In [ ]:
import torch
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import time

import datasets.mvtec as mvtec

from feature_extraction import Resnet18Features, concatenateLayers, extractEmbeddingVectors, extractEmbeddingVectorsDataloader
from distribution_fitting import getMeanCov
from distance_calculation import calculateScoreMaps

from scipy.ndimage import gaussian_filter



In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
device

In [3]:
class_name = 'bottle'
data_path = '../mvtec_dataset/'
dataset = mvtec.MVTecDataset(data_path, class_name=class_name, is_train=True)
dataloader = DataLoader(dataset, batch_size=32, pin_memory=True)
x, y, mask = next(iter(dataloader))

In [4]:
resnet18_features = Resnet18Features(device)

In [5]:
batch_embedding_vectors = extractEmbeddingVectors(resnet18_features, x)

In [6]:
batch_embedding_vectors.shape

torch.Size([32, 448, 56, 56])

In [7]:
embedding_vectors = extractEmbeddingVectorsDataloader(resnet18_features, dataloader)


Feature extraction: 100%|██████████| 7/7 [00:15<00:00,  2.21s/it]


In [8]:
embedding_vectors.shape

torch.Size([209, 448, 56, 56])

In [9]:
mean, cov_inv = getMeanCov(embedding_vectors)

Calculating mean


Calculating covariance: 100%|██████████| 3136/3136 [00:20<00:00, 152.06it/s]


In [10]:
mean.shape

torch.Size([448, 3136])

In [11]:
cov_inv.shape

torch.Size([448, 448, 3136])

In [17]:
tic = time.perf_counter()
score_map = calculateScoreMaps(mean, cov_inv, embedding_vectors[0].unsqueeze(0))
print(time.perf_counter()-tic)


0.24423491300001388


In [18]:
score_map.shape

torch.Size([1, 56, 56])